In [ ]:
!pip install tensorflow-hub
!pip install tensorflow-datasets

In [19]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [22]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)
train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [23]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = ""

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok) 
tokenizer.fit_on_texts(training_sentences) 
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences) 
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [24]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [25]:
# Task of Day 4, Earlystopping with Patience 3

from tensorflow.keras.callbacks import EarlyStopping

earlystop_callback = EarlyStopping(
  monitor='val_loss', min_delta=0.0001, patience=3)

In [27]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final), callbacks=[earlystop_callback])
model.evaluate(testing_padded)

Epoch 1/10
782/782 [==============================] - 81s 98ms/step - loss: 0.5663 - accuracy: 0.6839 - val_loss: 0.3952 - val_accuracy: 0.8273
Epoch 2/10
782/782 [==============================] - 76s 98ms/step - loss: 0.3372 - accuracy: 0.8592 - val_loss: 0.3627 - val_accuracy: 0.8394
Epoch 3/10
782/782 [==============================] - 76s 98ms/step - loss: 0.2671 - accuracy: 0.8969 - val_loss: 0.3990 - val_accuracy: 0.8338
Epoch 4/10
782/782 [==============================] - 76s 97ms/step - loss: 0.2165 - accuracy: 0.9201 - val_loss: 0.4066 - val_accuracy: 0.8374
Epoch 5/10
782/782 [==============================] - 2s 1ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


[0.0, 0.0]

Early Stopping stopped the training after 3 epochs of increase in lose thus saving from overfitting.